In [272]:
import pandas as pd
import numpy as np
from random import randint, sample
from urllib.parse import urlparse
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from itertools import combinations
from fuzzywuzzy import process, fuzz

In [195]:
all_products = [p.split(',') for p in open('data/product_urls.csv', 'r').readlines()]

In [196]:
offers = [list(set([urlparse(url).path.replace('-', ' ').replace('/', ' ').replace('\n', '') for url in urls])) for urls in all_products]

In [197]:
def combine_same_product(offer_list):
    comb = list(combinations(offer_list, 2))
    return [{'offer1': c[0], 'offer2': c[1], 'label': 1} for c in comb]

In [218]:
def combine_diff_product(offer_list):
    comb = list(combinations(offer_list, 2))
    return [{'offer1': c[0], 'offer2': c[1], 'label': 0} for c in comb]

In [219]:
spc = [combine_same_product(p) for p in offers]
random_diff_offers = [p[randint(0, len(p)-1)] for p in offers]
dpc = sample(combine_diff_product(random_diff_offers), 400)

In [229]:
all_combs = []
for comb in spc:
    for p in comb:
        all_combs.append(p)
        
for comb in dpc:
    all_combs.append(comb)

In [230]:
X = []
y = []
for item in all_combs:
    X.append([item['offer1'], item['offer2']])
    y.append(item['label'])

In [231]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [238]:
vec = TfidfVectorizer(tokenizer=word_tokenize, lowercase=True)
vecs = [vec.fit_transform(x) for x in X_train]

In [244]:
terms = []
for row in offers:
    for item in row:
        terms.append(item)

In [245]:
tdidf = TfidfVectorizer(tokenizer=word_tokenize, lowercase=True)

In [246]:
tdidf.fit_transform(terms)

<392x1543 sparse matrix of type '<class 'numpy.float64'>'
	with 4957 stored elements in Compressed Sparse Row format>

In [282]:
for i in range(len(X_train)):
    f = fuzz.token_sort_ratio(*X_train[i])
    if y_train[i] == 1:
        print("offer1: {}\n offer2: {}\n similarity: {}\n".format(X_train[i][0], X_train[i][1], f))

offer1:  BelezaSaude cuidadosfemininos modelaroreseescovasrotativas Modelador De Cachos Relaxbeauty Ana Hickmann   Titanium Control Ceramica 90 A 220 C 8478010.html
 offer2:  modelador de cachos relaxbeauty ana hickmann titanium control ceramica 90 a 220 c relaxbeauty p 8530850 cp pdpb 
 similarity: 72

offer1:  TelefoneseCelulares Smartphones Android smartphone samsung galaxy s8 plus preto com 64gb tela 6 2 android 7 0 4g reconhecimento de iris camera 12mp dual pixel e processador octa core 11482056.html
 offer2:  smartphone samsung galaxy s8 sm g955fd prata dual chip android 7 0 4g wi fi com tela infinita de 62 5452395 pr
 similarity: 54

offer1:  produto 121285989 fone de ouvido philips preto shl3060bk 00
 offer2:  fone de ouvido shl3060 philips p 2119827 ea eafn 
 similarity: 65

offer1:  Informatica AcessoriosePerifericos Mouses Microsoft Mouse Optico 1850 sem Fio U7Z 00038 Vermelho 4942033.html
 offer2:  produto 120648899 mouse microsoft 1850 wireless rosa
 similarity: 37

offer1